In [30]:
import os
import pandas as pd
from copy import deepcopy
from collections import defaultdict
import geopandas as gp
from matplotlib import pyplot as plt
from pam import parse

path_to_repo = '/Users/.../PythonProjects/pam'

## Load Data
Here we load simple travel diary data of London commuters. This is a very simple 0.1% sample of data about work and education commutes from the 2011 census. Because we're sharing this date - we've aggregated locations to borough level and randomized personal attributes - so don't get too excited about the results.

The data is available in `pam/example_data`.

In [2]:
trips = pd.read_csv(os.path.join(path_to_repo, 'example_data', 'example_travel_diaries.csv'))
attributes = pd.read_csv(os.path.join(path_to_repo, 'example_data' , 'example_attributes.csv'))
attributes.set_index('pid', inplace=True)

In [3]:
attributes.head()

,gender,job,occ,inc
pid,,,,
census_0,female,work,white,low
census_1,female,work,white,low
census_2,male,work,blue,high
census_3,male,work,blue,low
census_4,male,work,blue,medium


In [4]:
trips.head()

,uid,pid,hid,seq,hzone,ozone,dzone,purp,mode,tst,tet,freq
0,0,census_0,census_0,0,Harrow,Harrow,Camden,work,pt,444,473,1000
1,1,census_0,census_0,1,Harrow,Camden,Harrow,work,pt,890,919,1000
2,2,census_1,census_1,0,Greenwich,Greenwich,Tower Hamlets,work,pt,507,528,1000
3,3,census_1,census_1,1,Greenwich,Tower Hamlets,Greenwich,work,pt,1065,1086,1000
4,4,census_2,census_2,0,Croydon,Croydon,Croydon,work,pt,422,425,1000


In [5]:
trips['purp'].unique(), trips['mode'].unique()

(array(['work', 'education'], dtype=object),
 array(['pt', 'car'], dtype=object))

## Imagine we had better data

Conjure up additional data to aid demonstration.

We add:
- `age` to person attributes
- activities: 
    - `leisure`
    - `health`
    - `shopping`

**Randomly.**

In [6]:
import numpy as np

In [7]:
attributes['age'] = [int(a) for a in np.random.normal(40.5, 10, len(attributes))]

In [17]:
def print_simple_stats(population):
    """
    Print some simple population statistics.
    """
    time_at_home = 0
    travel_time = 0 
    low_income_central_trips = 0
    high_income_central_trips = 0
    
    for hid, hh in population.households.items():
        for pid, person in hh.people.items():
            freq = person.freq
            
            for p in person.plan:
                if p.act == 'travel':
                    duration = p.duration.seconds * freq / 3600
                    travel_time += duration
                    
                    if p.end_location.area == "Westminster,City of London":
                        if person.attributes['inc'] == "low":
                            low_income_central_trips += freq
                            
                        elif person.attributes['inc'] == "high":
                            high_income_central_trips += freq
                    
                else:  # activity
                    if p.act == 'home':
                        duration = p.duration.seconds * freq / 3600
                        time_at_home += duration
                        
    print(f"Population total time at home: {time_at_home/1000000:.2f} million hours")
    print(f"Population total travel time: {travel_time/1000000:.2f} million hours")
    print(f"Low income trips to Central London: {low_income_central_trips} trips")
    print(f"High income trips to Central London: {high_income_central_trips} trips")

## Create the population

In [8]:
population = parse.load_travel_diary(trips, attributes)

In [9]:
population.households['census_0'].people['census_0'].plan.print()

0:	Activity(0 act:home, location:Harrow, time:00:00:00 --> 07:24:00, duration:7:24:00)
1:	Leg(0 mode:pt, area:Harrow --> Camden, time:07:24:00 --> 07:53:00, duration:0:29:00)
2:	Activity(1 act:work, location:Camden, time:07:53:00 --> 14:50:00, duration:6:57:00)
3:	Leg(1 mode:pt, area:Camden --> Harrow, time:14:50:00 --> 15:19:00, duration:0:29:00)
4:	Activity(2 act:home, location:Harrow, time:15:19:00 --> 23:59:00, duration:8:40:00)


In [25]:
population.households['census_0'].people['census_0'].attributes

{'gender': 'female', 'job': 'work', 'occ': 'white', 'inc': 'low', 'age': 41}

In [26]:
print_simple_stats(population)

Population total time at home: 49.46 million hours
Population total travel time: 1.58 million hours
Low income trips to Central London: 229000 trips
High income trips to Central London: 246000 trips


# PAM Simple Policies

In [19]:
from pam import modify

def apply_policies(population, policies: list):

    new_population = deepcopy(population) 
    for hid, household in new_population.households.items():
        for policy in policies:
            policy.apply_to(household)
    return new_population

## Household Quarantine

Probabilistically apply quarantine to a household (remove all activities - stay at home) 

- if probability per person
- if probability per household

In [12]:
policy_household_quarantine_per_household = modify.HouseholdQuarantined(probability=0.01, person_based=False)

In [21]:
print_simple_stats(apply_policies(population, [policy_household_quarantine_per_household]))

Population total time at home: 49.69 million hours
Population total travel time: 1.56 million hours
Low income trips to Central London: 228000 trips
High income trips to Central London: 244000 trips


In [16]:
policy_household_quarantine_per_person = modify.HouseholdQuarantined(probability=0.01, person_based=True)

In [22]:
print_simple_stats(apply_policies(population, [policy_household_quarantine_per_person]))

Population total time at home: 49.60 million hours
Population total travel time: 1.57 million hours
Low income trips to Central London: 228000 trips
High income trips to Central London: 246000 trips


## Remove Higher Education

Remove all education activity for persons over age of 17

In [13]:
policy_remove_higher_education = modify.RemoveActivity(['education'], probability=0.95)

## Remove Education

Probabilistically remove education activities from a person

In [14]:
policy_remove_education = modify.RemoveActivity(['education'], probability=0.95)

## Remove Leisure Activities

Remove all leisure activities

## Remove Health Activities

Probabilistically remove individual health activities from a person

## Unemployment/Furlough

Probabilistically remove all work activities from a person

## Work from Home

Probabilistically remove all work activities from a person

## Reduced Work Activity

Probabilistically remove individual work activities from a person

## Remove Shopping 

Probabilistically remove individual shopping activities from a person

In [15]:
policy2 = modify.PersonStayAtHome(probability=0.1)
policy4 = modify.RemoveActivity(['education', 'work'], probability=0.9)